In [1]:
"""Code assumes the ability to train using a GPU with CUDA.
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from advertorch.attacks import GradientSignAttack, CarliniWagnerL2Attack, PGDAttack
import matplotlib.pyplot as plt
import utils.adv_ex_utils as aus
import utils.interp_generators as igs
import utils.utils as utils
from utils.models import LeNet
from utils.data_loaders import DataLoader

# makes default tensor a CUDA tensor so GPU can be used
torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = 2
torch.cuda.set_device(device)

### Load data

In [2]:
dl = DataLoader(dataset='MNIST')
datapreprocess = dl.data_preprocess
train_loader = dl.train_loader
test_loader = dl.test_loader
tr_batch_size = dl.tr_batch_size
te_batch_size = dl.te_batch_size

### Define train and test functions 

In [3]:
def train():
    net.train()
    
    for batch_idx, (samples, labels) in enumerate(train_loader):
        # sends to GPU, i.e. essentially converts from torch.FloatTensor to torch.cuda.FloatTensor
        samples, labels = samples.to(device), labels.to(device)
        
        # expand dataset with adversarial examples if adversary specified
        if adversary != None:
            adv_samples, adv_labels = aus.generate_adv_exs(samples, labels, adversary)
            samples, labels = torch.cat([samples, adv_samples], 0), torch.cat([labels, adv_labels], 0)
                
        optimizer.zero_grad()
        
        output = net(samples)
        
        loss = utils.my_loss(output, labels, net=net, optimizer=optimizer,
                             alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples, 
                             bp_mat=tr, alpha_ir1=alpha_ir1, alpha_ir2=alpha_ir2)
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            j = utils.avg_norm_jacobian(net, samples, output.shape[1], tr, for_loss=False)
            i,_ = utils.norm_diff_interp(net, samples, labels)
            print(f'\tLoss: {loss.item():.6f} Average norm of Jacobian: {j:6f} Norm of difference in interpretations: {i:6f}')

In [4]:
def test():
    net.eval()
    test_loss = 0
    correct = 0
    
    for samples, labels in test_loader:
        samples, labels = samples.to(device), labels.to(device)
        output = net(samples)
        test_loss += utils.my_loss(output, labels, net=net, 
                             alpha_wd=alpha_wd, alpha_jr=alpha_jr, x=samples, 
                             bp_mat=tr, alpha_ir1=alpha_ir1, alpha_ir2=alpha_ir2).item()
        # output is a tensor, .data retrieves its data, max returns the index of the highest valued element
        preds = output.data.max(1, keepdim=True)[1]
        correct += preds.eq(labels.data.view_as(preds)).sum().item()
                
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * float(correct / len(test_loader.dataset))
    
    print(f'\tTest set accuracy: ({test_accuracy:.2f}%)')

### Training with interpretation regularization

In [10]:
# training details
n_epochs = 30
log_interval = 200
training_round = 2
torch.manual_seed(training_round)
adversary=None

# loss hyperparameters
alpha_wd = 0
alpha_jr = 0
alpha_ir1 = 50
alpha_ir2 = .000001

# instantiate model and optimizer
learning_rate = 0.01
momentum = 0.9
net = LeNet()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)
lr_decayer = StepLR(optimizer, step_size=10, gamma=0.1)

# make model CUDA enabled and define GPU/device to use
net.cuda()

tr = utils.bp_matrix(tr_batch_size, 10)
te = utils.bp_matrix(te_batch_size, 10)
for epoch in range(1, n_epochs + 1):
    print(f'Epoch #{epoch}')
    train()
    test()
    lr_decayer.step()

Epoch #1
	Loss: 18.742104 Average norm of Jacobian: 0.015210 Norm of difference in interpretations: 0.330283
	Loss: nan Average norm of Jacobian:    nan Norm of difference in interpretations:    nan


KeyboardInterrupt: 

In [57]:
torch.save(net.state_dict(), f'trained_models/interp_reg_tests/jr{}_ir1{}_ir2{}')